In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
data = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD.csv')
data.index = data['PatientID']
# drop last 2 columns
data = data.drop(data.columns[-2:], axis=1)
data

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
PatientID,,,,,,,,,,,,,,,,,,,,,
TCGA-A6-6650,-0.457220,0.851885,0.540813,-0.423622,0.714115,-0.564329,1.607602,0.543431,-0.870704,0.980508,...,0.673500,0.445102,0.272053,0.577840,-0.313309,0.463409,0.216033,1.875605,-0.811024,-1.250726
TCGA-A6-6142,-0.387709,0.730046,0.720710,-0.723050,0.841878,-0.463233,1.442049,0.573482,-0.817522,1.146394,...,0.287993,0.639623,0.308657,0.696497,-0.164076,0.249183,0.075877,2.179679,-0.814272,-1.311477
TCGA-5M-AAT4,-0.430112,0.766393,0.768542,-0.344121,0.560576,-0.597754,1.708495,0.419003,-0.934103,1.152953,...,0.578889,0.539531,0.399675,0.471158,-0.089670,0.701613,0.178599,1.802830,-0.844305,-1.300183
TCGA-A6-2676,-0.417006,0.964753,0.525541,-0.572356,0.777732,-0.462124,1.476457,0.640170,-0.722137,1.066126,...,0.475959,0.557490,0.094860,0.659872,-0.243587,0.122108,0.107965,2.146043,-0.946326,-1.345658
TCGA-A6-6652,-0.276354,0.766071,0.612194,-0.596442,0.814464,0.016801,1.598807,0.929635,-0.914753,1.071625,...,0.395806,0.545849,0.290805,0.229333,-0.536335,0.017757,-0.074384,2.026599,-0.776906,-0.934953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-G4-6321,-0.359843,0.929950,0.543955,-0.350527,0.404216,-0.566966,1.688820,0.196331,-1.050643,1.162059,...,0.802867,0.499934,0.404179,0.325667,-0.166769,0.612408,0.197617,1.686868,-0.917694,-1.251037
TCGA-RU-A8FL,-0.480253,0.931134,0.455165,-0.714430,0.685353,-0.373306,1.511218,0.747226,-0.652520,0.907837,...,0.179997,0.744851,0.062442,0.249245,-0.573766,-0.159084,-0.157464,2.173445,-0.916892,-1.098334
TCGA-G4-6303,-0.397405,0.793659,0.682602,-0.526544,0.733674,-0.456034,1.593480,0.500975,-0.982290,1.068642,...,0.514841,0.515760,0.324934,0.458768,-0.172873,0.498273,0.168238,1.823696,-0.805411,-1.238766


In [3]:
data_target = pd.read_csv('~/Desktop/project_data_new/target_768_avg_expanded.csv')

In [4]:
data_target.index = data_target['Unnamed: 0']
data_target = data_target.drop(['Unnamed: 0'], axis = 1)
# only keep the columns with category in the name
data_target = data_target.loc[:, data_target.columns.str.contains('category')]
data_target

,category_tnk_1,category_tnk_2,category_tnk_3,category_tnk_4,category_tnk_5,category_tnk_6,category_tnk_7,category_tnk_8,category_tnk_9,category_tnk_10,...,category_stromal_62,category_stromal_63,category_stromal_64,category_stromal_65,category_stromal_66,category_stromal_68,category_stromal_73,category_stromal_78,category_stromal_80,category_stromal_85
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
TCGA-A6-6650,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-A6-6142,1,1,0,0,0,0,0,0,-1,0,...,1,1,1,1,1,-1,0,0,-1,-1
TCGA-5M-AAT4,-1,-1,-1,0,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-A6-2676,1,1,1,0,1,1,1,1,1,1,...,0,-1,0,-1,0,1,0,1,1,1
TCGA-A6-6652,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-G4-6321,1,1,1,-1,1,1,1,1,-1,1,...,0,0,-1,1,1,-1,1,1,-1,-1
TCGA-RU-A8FL,-1,-1,-1,1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,-1,-1,1,1
TCGA-G4-6303,-1,0,-1,-1,0,0,0,-1,-1,1,...,1,1,1,0,1,-1,0,0,-1,-1


In [5]:
data = data[data.index.isin(data_target.index)]
data.shape

(449, 768)

# b_12

In [6]:
X = data.values
# Extract the target column for category_b_12
Y_category_b_12 = data_target['category_b_12'].values

#### 100 features


In [9]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
X_selected = rfe.fit_transform(X, Y_category_b_12)


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint

# X = data.values
X = X_selected
# Extract the target column for category_b_12
Y_category_b_12 = data_target['category_b_12'].values

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Define the hyperparameter distribution (use distributions for random search)
param_dist = {
    'n_estimators': randint(100, 200),            # Fewer trees may generalize better
    'max_depth': randint(20, 40),                 # Lower depth can prevent overfitting
    'min_samples_split': randint(10, 20),         # Increase split to limit tree growth
    'min_samples_leaf': randint(1, 5),            # Require more samples per leaf
    'max_features': ['sqrt', 'log2']
}


# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_rf = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_rf, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
print("Classification Report for category_b_12 (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_rf.predict(X_test)
print("Classification Report for category_b_12 (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters: {'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 13, 'n_estimators': 123}
Cross-validation F1 scores on training set: [0.55619445 0.4369223  0.49958522 0.37427536 0.5233637 ]
Average cross-validation F1 score on training set: 0.4780682076558113
Classification Report for category_b_12 (Train Set):
              precision    recall  f1-score   support

          -1       1.00      0.98      0.99       117
           0       0.97      0.99      0.98       122
           1       0.99      0.98      0.99       120

    accuracy                           0.99       359
   macro avg       0.99      0.99      0.99       359
weighted avg       0.99      0.99      0.99       359

Classification Report for category_b_12 (Test Set):
              precision    recall  f1-score   support

          -1       0.62      0.70      0.66        33
           0       0.74      0.52      

#### 150 features

In [32]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
X = data.values
rf = RandomForestClassifier()
rfe = RFE(rf, n_features_to_select=150)  # Choose top 100 features
X = rfe.fit_transform(X, Y_category_b_12)


In [33]:
# save X as Top_100_features_REF
np.savetxt("Top_150_features_REF.csv", X, delimiter=",")

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint

# X = data.values
X = X_selected
# Extract the target column for category_b_12
Y_category_b_12 = data_target['category_b_12'].values

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Define the hyperparameter distribution (use distributions for random search)
param_dist = {
    'n_estimators': randint(100, 200),            # Fewer trees may generalize better
    'max_depth': randint(20, 40),                 # Lower depth can prevent overfitting
    'min_samples_split': randint(10, 20),         # Increase split to limit tree growth
    'min_samples_leaf': randint(1, 5),            # Require more samples per leaf
    'max_features': ['sqrt', 'log2']
}


# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_rf = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_rf, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
print("Classification Report for category_b_12 (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_rf.predict(X_test)
print("Classification Report for category_b_12 (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters: {'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 13, 'n_estimators': 123}
Cross-validation F1 scores on training set: [0.55619445 0.4369223  0.49958522 0.37427536 0.5233637 ]
Average cross-validation F1 score on training set: 0.4780682076558113
Classification Report for category_b_12 (Train Set):
              precision    recall  f1-score   support

          -1       1.00      0.98      0.99       117
           0       0.97      0.99      0.98       122
           1       0.99      0.98      0.99       120

    accuracy                           0.99       359
   macro avg       0.99      0.99      0.99       359
weighted avg       0.99      0.99      0.99       359

Classification Report for category_b_12 (Test Set):
              precision    recall  f1-score   support

          -1       0.62      0.70      0.66        33
           0       0.74      0.52      

In [37]:
X = pd.read_csv('Top_150_features_REF.csv', header=None)


#### 50 features

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint

# X = data.values
X 
# Extract the target column for category_b_12
Y_category_b_12 = data_target['category_b_12'].values

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Define the hyperparameter distribution (use distributions for random search)
param_dist = {
    'n_estimators': randint(100, 200),            # Fewer trees may generalize better
    'max_depth': randint(20, 40),                 # Lower depth can prevent overfitting
    'min_samples_split': randint(10, 20),         # Increase split to limit tree growth
    'min_samples_leaf': randint(1, 5),            # Require more samples per leaf
    'max_features': ['sqrt', 'log2']
}


# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=50, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_rf = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_rf, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
print("Classification Report for category_b_12 with 150 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_rf.predict(X_test)
print("Classification Report for category_b_12 150 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters: {'max_depth': 37, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 11, 'n_estimators': 105}
Cross-validation F1 scores on training set: [0.58516022 0.47737382 0.47200305 0.36557754 0.53521127]
Average cross-validation F1 score on training set: 0.4870651797710435
Classification Report for category_b_12 with 150 features (Train Set):
              precision    recall  f1-score   support

          -1       1.00      0.98      0.99       117
           0       0.97      1.00      0.98       122
           1       1.00      0.98      0.99       120

    accuracy                           0.99       359
   macro avg       0.99      0.99      0.99       359
weighted avg       0.99      0.99      0.99       359

Classification Report for category_b_12 150 features (Test Set):
              precision    recall  f1-score   support

          -1       0.59      0.61      0.60        33
        

In [7]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
X = data.values
rf = RandomForestClassifier()
rfe = RFE(rf, n_features_to_select=50)  # Choose top 100 features
X = rfe.fit_transform(X, Y_category_b_12)


In [8]:
# save 50 
np.savetxt("Top_50_features_REF.csv", X, delimiter=",")

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from scipy.stats import randint

# X = data.values
X 
# Extract the target column for category_b_12
Y_category_b_12 = data_target['category_b_12'].values

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Define the hyperparameter distribution (use distributions for random search)
param_dist = {
    'n_estimators': randint(100, 150),            # Fewer trees may generalize better
    'max_depth': randint(8, 15),                 # Lower depth can prevent overfitting
    'min_samples_split': randint(20, 25),         # Increase split to limit tree growth
    'min_samples_leaf': randint(10, 15),            # Require more samples per leaf
    'max_features': ['sqrt', 'log2']
}


# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=80, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_rf = random_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", random_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_rf, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
print("Classification Report for category_b_12 with 150 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_rf.predict(X_test)
print("Classification Report for category_b_12 150 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best Hyperparameters: {'max_depth': 14, 'max_features': 'sqrt', 'min_samples_leaf': 13, 'min_samples_split': 21, 'n_estimators': 132}
Cross-validation F1 scores on training set: [0.54206103 0.51532835 0.46645022 0.45846601 0.49609658]
Average cross-validation F1 score on training set: 0.49568043746357643
Classification Report for category_b_12 with 150 features (Train Set):
              precision    recall  f1-score   support

          -1       0.86      0.80      0.83       117
           0       0.81      0.86      0.84       122
           1       0.87      0.88      0.87       120

    accuracy                           0.85       359
   macro avg       0.85      0.85      0.85       359
weighted avg       0.85      0.85      0.85       359

Classification Report for category_b_12 150 features (Test Set):
              precision    recall  f1-score   support

          -1       0.64      0.64      0.64        33
      

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

# Assuming X and Y_category_b_12 are already defined
# X = data.values
# Y_category_b_12 = data_target['category_b_12'].values

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_category_b_12, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Define the hyperparameter grid (specific values for each hyperparameter)
param_grid = {
    'n_estimators': list(range(120, 141, 2)),        # Every integer between 100 and 150 with step 10
    'max_depth': list(range(8, 16)),                  # Every integer between 8 and 15
    'min_samples_split': list(range(20, 26)),         # Every integer between 20 and 25
    'min_samples_leaf': list(range(10, 16)),          # Every integer between 10 and 15
    'max_features': ['sqrt', 'log2']                  # Options for max features
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1)
grid_search.fit(X_train, Y_train)

# Get the best model from Grid Search
best_rf = grid_search.best_estimator_

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Cross-validation on the training set with the best model
cv_scores = cross_val_score(best_rf, X_train, Y_train, cv=5, scoring='f1_weighted')
print("Cross-validation F1 scores on training set:", cv_scores)
print("Average cross-validation F1 score on training set:", np.mean(cv_scores))

# Fit the model on the training data for the final evaluation on test and train sets
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
print("Classification Report for category_b_12 with 150 features (Train Set):")
print(classification_report(Y_train, Y_train_pred))

# Evaluate on the test set
Y_test_pred = best_rf.predict(X_test)
print("Classification Report for category_b_12 150 features (Test Set):")
print(classification_report(Y_test, Y_test_pred))


Fitting 5 folds for each of 6336 candidates, totalling 31680 fits
Best Hyperparameters: {'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 13, 'min_samples_split': 20, 'n_estimators': 134}
Cross-validation F1 scores on training set: [0.54206103 0.51532835 0.46645022 0.45846601 0.52420446]
Average cross-validation F1 score on training set: 0.5013020136103525
Classification Report for category_b_12 with 150 features (Train Set):
              precision    recall  f1-score   support

          -1       0.86      0.81      0.84       117
           0       0.81      0.86      0.84       122
           1       0.88      0.88      0.88       120

    accuracy                           0.85       359
   macro avg       0.85      0.85      0.85       359
weighted avg       0.85      0.85      0.85       359

Classification Report for category_b_12 150 features (Test Set):
              precision    recall  f1-score   support

          -1       0.64      0.64      0.64        33
    